In [23]:
import os
import math
from tarefa02.gcrf import Gcrf, Point, Edge, Face

In [24]:
gc = Gcrf()

In [25]:
p = Point(1, 2, 3)
print(p)

(1, 2, 3)


In [26]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)

e = Edge(p1, p2)

print(e)

[(1, 2, 3) -> (4, 5, 6)]


In [27]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)
p3 = Point(7, 8, 9)

e1 = Edge(p1, p2)
e2 = Edge(p2, p3)
e3 = Edge(p3, p1)

f = Face(e1, e2, e3)
f_ = Face(e3, e1, e2)
print(f)

{[(1, 2, 3) -> (4, 5, 6)], [(4, 5, 6) -> (7, 8, 9)], [(7, 8, 9) -> (1, 2, 3)]}


In [28]:
def checa_face(F, f):
    for face in F:
        if f == face:
            return True
    return False

In [29]:
p1 = Point(0,-1,0)
p2 = Point(0, 0, 0)
p3 = Point(0,0,1)
a = Point(1, 0,0)
b = Point(-1,0,0)
print(gc.signed_volume(p1,p2,p3,a))
print(gc.signed_volume(p1,p2,p3,b))

0.16666666666666666
-0.16666666666666666


In [30]:
def PivotAroundEdge(e, P):
    minAngle = 2*math.pi
    candidate = None
    normal = gc.prod_vetorial(e.p1, e.p2)
#     print("NORMAL:", normal)
    for i in range(len(P)):
#         print("P[i]",P[i])
        angle = gc.angulo(normal, P[i])
        if angle < minAngle:
            minAngle = angle
            candidate = i
    return P[candidate]

In [31]:
def BottomMostLeftMostBackMost(P):
    ys = [p.y for p in P]
    min_value = min(ys)
    min_index = ys.index(min_value)
    return P[min_index]

In [32]:
def FindEdgeOnHull(P):
    p = BottomMostLeftMostBackMost(P)
    q = Point(p.x + 1, p.y, p.z)
    q = PivotAroundEdge(Edge(p, q), P)
    
    return Edge(p, q)

In [33]:
def FindTriangleOnHull(P):
    e = FindEdgeOnHull(P)
    r = PivotAroundEdge(e, P)
    e2 = Edge(e.p2, r)
    e3 = Edge(r, e.p1)
    f = Face(e, e2, e3)
    return f

In [34]:
def NotProcessed(E, e):
#     print("Aresta no Not Processed",e)
    for i in E:
        if e == i:
            return False
    return True

In [45]:
def GiftWrap(P):
    t = FindTriangleOnHull(P)
#     print("Triangulo encontrado:", t)
    Q = [t.e1, t.e2, t.e3]
    H = [t]
    Processed = []

    while not len(Q) == 0:
        e = Q.pop()
        e = Edge(e.p2, e.p1)
        #print("Aresta a ser processada:", e)
        if NotProcessed(Processed, e):
            #print("Aresta {} não foi processada, entrando no IF".format(e))
            q = PivotAroundEdge(e, P)
            #print("Resultado do PivotAroundEdge:", q)
            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
            #print("Nova face:", t)
            face_no_hull = checa_face(H, t)
            if not face_no_hull:
                H.append(t)
                #print("{} entrou no Hull".format(t))
#             e_ = Edge(e.p2, e.p1)
            Q.extend([t.e1, t.e2, t.e3])
            #print("Q agora possui as arestas:")
            #for q in Q: print(q)
            Processed.append(e)
            #print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
            #for aresta in Processed: print(aresta)
        #else: print("Aresta JÁ PROCESSADA")
    return H

In [36]:
# p1 = Point(-3, 0, 0)
# p2 = Point(0, -3, 0)
# p3 = Point(0, 2, 0)
# p4 = Point(0, 0, 3)
# p5 = Point(0, 0, -4)
# p6 = Point(4, 0, 0)
# p7 = Point(0, 0, 0)

p1 = Point(-1, 0, 0)
p2 = Point(0, -1, 0)
p3 = Point(0, 1, 0)
p4 = Point(0, 0, 1)
p5 = Point(0, 0, -1)
p6 = Point(1, 0, 0)

P = [p1, p2, p3, p4, p5, p6]

H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))

Triangulo encontrado: {[(0, -1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0)]}
{[(0, -1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0)]}
{[(-1, 0, 0) -> (0, -1, 0)], [(0, -1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (-1, 0, 0)]}
2


In [37]:
def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                    i1 = j
                if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                    i2 = j
                if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')


In [38]:
def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                i1 = j
            if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                i2 = j
            if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                i3 = j
                
        print("f {} {} {}".format(i1+1, i2+1, i3+1))

In [39]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            line = line.replace('v', '')
            line_ = line.split()
            points.append(Point(float(line_[0]), float(line_[1]), float(line_[2])))
    return points

In [40]:
points = read_vertices_from_obj(r"C:\Users\romul\Documents\UFC\MESTRADO\Disciplinas\Geometria Computacional\super-nintendo-blend\p1.obj")
print(len(points))

348


In [46]:
nH = GiftWrap(points)
# for h in nH:
#     print(h)
print(len(nH))

78


In [42]:
print_faces(H, P)
write_faces(H, P)

f 2 5 1
f 1 2 5


In [47]:
write_faces(nH, points)
# print_faces(nH, points)
